In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
pip install PyPDF2

     |████████████████████████████████| 77 kB 2.9 MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-py3-none-any.whl size=61100 sha256=093074b76e42f506dac70064ceb027bfcce541e945b62f24cdd393c5dfee07e0
  Stored in directory: /root/.cache/pip/wheels/80/1a/24/648467ade3a77ed20f35cfd2badd32134e96dd25ca811e64b3
Successfully built PyPDF2


In [ ]:
import PyPDF2
import re
from gensim.models import Word2Vec
from gensim.models import FastText
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.parsing.preprocessing import STOPWORDS, RE_WHITESPACE
from gensim.corpora.dictionary import Dictionary
from gensim.models import KeyedVectors

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
model_w2v = KeyedVectors.load_word2vec_format('drive/My Drive/Colab Notebooks/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [ ]:
file = PyPDF2.PdfFileReader('drive/My Drive/Colab Notebooks/dataset.pdf')
Numpages=file.getNumPages()
text = ""
for j in range(0,Numpages):
    PageObj = file.getPage(j)
    text = text + " " + PageObj.extractText()

In [ ]:
#paragraphs = re.split ('\.|\?|\!', text) # Sentence break
paragraphs = re.split ('\.\n', text) # Paragraph break
paragraphs = [re.sub('[^-\.!\?,;a-zA-Z0-9]+', ' ', paragraph.strip()) for paragraph in paragraphs]
original_para = paragraphs.copy()

In [ ]:
lm = WordNetLemmatizer()
#paragraphs = [re.sub('[^-a-zA-Z0-9]+', ' ', paragraph.lower()) for paragraph in paragraphs]
for paragraph in paragraphs:
    paragraph = [lm.lemmatize(word) for word in paragraph if not word in stopwords.words('english')]

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)


In [ ]:
query = "Nehru and Indian Independence"

In [ ]:
# Method 1
query = query.lower ()
expanded_query = []
n = 5
for word in query:
  word = str(word)
  try:
    similar_vector = model_w2v.most_similar ([word], topn = n)
  except KeyError:
    similar_vector = []
  similar_vector.insert (0, (word, 1))
  expanded_query.append (similar_vector)

In [ ]:
# Method 2
query = query.lower ()
query = query.split (' ')
expanded_query = []
n = 10
for word in query:
  word = str(word)
  expanded_query.append(word)
  try:
    similar_vector = model_w2v.most_similar ([word], topn = n)
  except KeyError:
    similar_vector = []
  for element in similar_vector:
    similar_word = str(element[0]).lower().replace('_', ' ')
    expanded_query.append (similar_word)
expanded_query.append ('.')

In [ ]:
print (expanded_query)
print (query)

['nehru', 'gandhi', 'thakur', 'sonia', 'manmohan', 'aurangazeb', 'srk', 'mohan', 'anti hindu', 'sikh', 'chamcha', 'and', 'indian', 'india', 'american', 'indians', 'pakistani', 'british', 'chinese', 'sri lankan', 'pakistan', 'malaysian', 'tamil', 'independence', 'indepedence', 'autonomy', 'statehood', 'independance', 'independency', 'indepence', 'nationhood', 'de facto independence', 'secession', 'sovereignty', '.']
['nehru', 'and', 'indian', 'independence']


In [ ]:
import numpy as np
para_vectors = []
for paragraph in paragraphs:
  para_vector = np.array([0.0] * 300)
  para = paragraph.split()
  for word in para:
    word = str (word)
    try:
      word_vector = model_w2v[word]
    except KeyError:
      word_vector = [0.0] * 300
    para_vector += word_vector
  para_vector /= len(para) 
  para_vectors.append (para_vector)

In [ ]:
query_vector = np.array([0.0]*300)
for word in expanded_query:
  word = str (word)
  try:
    word_vector = model_w2v[word]
  except KeyError:
    word_vector = [0.0] * 300
  query_vector += word_vector
  query_vector /= len(expanded_query) 

In [ ]:
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [ ]:
similarity_vec = []
for i in range (len(paragraphs)):
  #similarity = cosine(query_vec, sbert_model.encode([paragraphs[i]])[0])
  similarity = cosine(query_vector, para_vectors[i])
  similarity_vec.append((i, similarity))
similarity_vec.sort (key = lambda x: -x[1])

In [ ]:
print (similarity_vec[:10])

[(697, 0.3493642454234973), (290, 0.3365584444908441), (287, 0.33554686057984073), (410, 0.3337287938418401), (644, 0.32617832425231), (567, 0.3243500587105559), (698, 0.32299826329795256), (636, 0.32141031622101324), (724, 0.314217199586565), (768, 0.3127424701175319)]


In [ ]:
max_relevant_para = 5

In [ ]:
for i in range (max_relevant_para):
  print (paragraphs [similarity_vec [i][0]])
  for i in range (5):
    print ()

He had been in touch with the Chinese nationalists in the course of the Indian freedom movement and he had great sympathy for the aspirations of the Chinese nation. In his first foreign policy speeches after India attained independence he had referred to Asia as a zone of peace conflict had started only with the intrusion of imperialism. With the emancipation of Asia peace was bound to be restored. In this optimistic spirit of antiimperialism he had also welcomed the victory of Mao Tse Tung and had hoped for peace and harmony with the great neighbour. When in 1950 the Chinese occupied Tibet which had long enjoyed a quasi-independent status Nehru quickly gave up those Tibetan outposts which India had inherited from the British. These outposts were imperialistic relics to him





The Brahmins and the ritual sovereignty of the kingThe precarious position of the king with regard to both the control of hiscentral area and his relations with his samantas called for a specific emphasis on th

array([ 1.15722656e-01, -1.29882812e-01,  6.59179688e-02,  1.75781250e-01,
       -1.43554688e-01,  6.88476562e-02,  4.88281250e-02, -1.26953125e-01,
        9.37500000e-02,  2.11914062e-01, -2.73437500e-01, -1.08398438e-01,
       -1.90429688e-01,  8.83789062e-02, -1.02539062e-01,  3.47656250e-01,
        2.04101562e-01, -4.85839844e-02, -2.07519531e-02,  9.32617188e-02,
        3.03955078e-02,  4.94384766e-03,  3.30078125e-01, -7.37304688e-02,
       -7.42187500e-02,  3.05175781e-02, -2.67578125e-01,  1.59179688e-01,
        8.00781250e-02,  1.97265625e-01,  6.98242188e-02,  1.24511719e-01,
       -1.51367188e-01,  2.35595703e-02,  5.90820312e-02,  3.61328125e-02,
        1.01074219e-01,  2.57812500e-01,  3.27148438e-02,  1.06445312e-01,
        1.80664062e-01,  5.76171875e-02,  2.51953125e-01,  3.58886719e-02,
        5.15136719e-02,  1.28906250e-01,  2.14843750e-02, -2.74658203e-02,
       -1.25976562e-01,  7.66601562e-02, -2.37304688e-01, -1.55029297e-02,
       -6.43920898e-03, -